In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install transformers

In [ ]:
!pip install SentencePiece

In [ ]:
import time, json,copy,torch

from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import time, json,copy,torch
def download_model():
    print("model loading......")
    model_name = "facebook/mbart-large-50-one-to-many-mmt"
    model = MBartForConditionalGeneration.from_pretrained('./mBart')
    tokenizer = MBart50Tokenizer.from_pretrained('./mBart')
    return model, tokenizer

def main():
    model, tokenizer = download_model()
    model = model.cuda()
    tokenizer.src_lang = "en_XX"
    data = open('../input/all-questions/vqa2AllQuestions.txt','r').readlines() # enter the path for input file
    data = [l.replace('\n','') for l in data]
    #start = time.time()
    ctr = 0
    out_file = open('./target_language_translation.txt','a') # Output file 
    #try:
    for st in range(0,len(data),32):
        end = len(data) if st + 32 >len(data) else st+32
        q = data[st:end]
        encoded_text = tokenizer(q, padding=True,truncation=True,return_tensors="pt")
        encoded_text['input_ids'] = encoded_text['input_ids'].cuda()
        encoded_text['attention_mask'] = encoded_text['attention_mask'].cuda()
        generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])   # enter the target language ID here, for example: "hi" means Hindi
        out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        print(end, " samples processesed.")
        for listitem in out:
            out_file.write('%s\n' % listitem)
        del encoded_text, out
        torch.cuda.empty_cache()
   # except:
    out_file.close()
    print("Done!!")

In [ ]:
model, tokenizer = download_model()

In [ ]:
model.save_pretrained('./mBart')
tokenizer.save_pretrained('./mBart')

In [ ]:
main()